In [1]:
import requests
import io
import os
import s3fs
import glob

import dask.dataframe as dd
import pandas as pd
import numpy as np

from dask import delayed
from distributed import Client
from distributed import progress, wait


fs = s3fs.S3FileSystem()
files = [f for f in fs.ls('nyc-tlc/trip data/') if '2016' in f and 'yellow' in f]

client = Client('tcp://ec2-34-200-221-251.compute-1.amazonaws.com:8786')
client.restart()

client

distributed.utils - ERROR - Timed out trying to connect to 'tcp://ec2-34-200-221-251.compute-1.amazonaws.com:8786' after 5 s: connect() didn't finish in time
Traceback (most recent call last):
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.6/site-packages/distributed/comm/core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.6/site-packages/tornado/gen.py", line 1055, in run
    value = future.result()
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.6/site-packages/tornado/concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.6/site-packages/distributed/utils.py", line 229, in f
    result[0] = yield make_coro()
  File "/home/mile

OSError: Timed out trying to connect to 'tcp://ec2-34-200-221-251.compute-1.amazonaws.com:8786' after 5 s: connect() didn't finish in time

In [3]:
import sys
counts = []

# Loop through years 2009-2017
for year in np.arange(2009, 2018):
    
    # Get only files pertaining to this year
    files = [f for f in fs.ls('nyc-tlc/trip data/') if str(year) in f and 'yellow' in f]
    
    # Process files in parallel. (client is asynchronous)
    for i, file in enumerate(files):
        
        # Extract year and month from filename
        _year, month = file[-11:-4].split('-')
        
        # Process data for current year and month
        df = dd.read_csv('s3://' + file, 
                         dtype='object',
                         error_bad_lines=False,
                         blocksize=int(128e6))
        df = df.map_partitions(clean_df)
        df = client.persist(df)
        
        # Yearly dataframe merging
        main_df = df if not i else main_df.append(df)
    
    # Write year's df to S3
    main_df.to_csv('s3://milesg-taxi-data-east/yellow-{year}-*.csv.gz'.format(year=year), compression='gzip')
    counts.append(main_df.passenger_count.count().compute())
    sys.stdout.write('\rYear: {} - Total {}'.format(_year, sum(counts)))
    
    # Clear from cluster memory
    client.cancel(main_df)


Year: 2017 - Total 1204768604

In [4]:
client.restart()

Client Scheduler: tcp://ec2-34-200-221-251.compute-1.amazonaws.com:8786 Dashboard: http://ec2-34-200-221-251.compute-1.amazonaws.com:8787,Cluster Workers: 16 Cores: 64 Memory: 269.26 GB


In [5]:
client.shutdown()